In [ ]:
!pip install ast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install lark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 KB 10.4 MB/s eta 0:00:00


In [ ]:
from lark import Lark

# Define the grammar
grammar = """
start: statement*

statement: loop | assign

loop: "for"  ID "in" range  block

assign: ID ":=" expression ";"

block: "{" statement* "}"

range: expression ".." expression

expression: ID | NUMBER | expression ( "+" | "-" ) expression

ID: /[a-zA-Z][a-zA-Z0-9]*/

NUMBER: /\d+/
%import common.WS
%ignore WS
"""

# Create the parser
parser = Lark(grammar)

code = "for i in 1..10 { x := 0; x := x + 1;}"

# Parse the code
tree = parser.parse(code)

# Find all loop statements
loops = tree.find_data('loop')

if loops:
    print("The code contains at least one loop.")
else:
    print("The code doesn't contain any loops.")

The code contains at least one loop.


In [ ]:
import ast

def check_nested_loops(tree):
    for node in ast.walk(tree):
        if isinstance(node, (ast.For, ast.While)):
            for child in ast.iter_child_nodes(node):
                if isinstance(child, (ast.For, ast.While)):
                    print("Nested loops found!")
                    return
    print("No nested loops found.")

code = '''
for i in range(10):
    for j in range(5):
        print(i * j)
'''

tree = ast.parse(code)
print(tree)
check_nested_loops(tree)


Nested loops found!


In [ ]:

import ast

code = '''
x = 2
y = x + 3
z = x * y

def my_func(a, b):
    c = a + b
    return c
'''

tree = ast.parse(code)

deps = {}
for node in ast.walk(tree):
    if isinstance(node, ast.Assign):
        for target in node.targets:
            if isinstance(target, ast.Name):
                dep = [n.id for n in ast.walk(node) if isinstance(n, ast.Name) and n.id != target.id]
                deps[target.id] = dep
print(deps)


{'x': [], 'y': ['x'], 'z': ['x', 'y'], 'c': ['a', 'b']}


In [ ]:
import ast

def check_variable_dependencies(tree):
    variables = set()
    for node in ast.walk(tree):
        if isinstance(node, ast.Name):
            variables.add(node.id)
        elif isinstance(node, (ast.For, ast.While, ast.FunctionDef, ast.ClassDef)):
            for child in ast.iter_child_nodes(node):
                if isinstance(child, ast.Assign):
                    for target in child.targets:
                        if isinstance(target, ast.Name):
                            variables.add(target.id)
    print("Variables:", variables)

code = '''
x = 2
y = x + 3
z = x * y

def my_func(a, b):
    c = a + b
    return c
'''
tree = ast.parse(code)
check_variable_dependencies(tree)


Variables: {'y', 'x', 'b', 'a', 'c', 'z'}


In [ ]:
import ast

code = '''
for i in range(10):
    for j in range(5):
        print(i * j)
'''

tree = ast.parse(code)

# Print the names of all nodes in the tree
for node in ast.walk(tree):
    print(node.__class__.__name__)


Module
For
Name
Call
For
Store
Name
Constant
Name
Call
Expr
Load
Store
Name
Constant
Call
Load
Name
BinOp
Load
Name
Mult
Name
Load
Load


In [ ]:
!pip install astroid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ast
from astroid import extract_node

def get_loop_count(tree):
    loop_count = 0
    for node in ast.walk(tree):
        if isinstance(node, (ast.For, ast.While)):
            loop_count += 1
    return loop_count

def get_variable_dependencies(tree):
    dependencies = {}
    for node in ast.walk(tree):
        if isinstance(node, ast.Assign):
            for target in node.targets:
                if isinstance(target, ast.Name):
                    dep = [n.id for n in ast.walk(node) if isinstance(n, ast.Name) and n.id != target.id]
                    dependencies[target.id] = dep
    return dependencies

solutions = [
    '''
x = 2
y = x + 3
z = x * y

def my_func(a, b):
    c = a + b
    return c
''',
    '''
for i in range(10):
    for j in range(5):
        print(i * j)
''',
    '''
while True:
    x = input("Enter a number: ")
    if x == "q":
        break
'''
]

dataset = []
for solution in solutions:
    tree = ast.parse(solution)
    loop_count = get_loop_count(tree)
    variable_dependencies = get_variable_dependencies(tree)
    dataset.append({'loop_count': loop_count, 'variable_dependencies': variable_dependencies})

print(dataset)


[{'loop_count': 0, 'variable_dependencies': {'x': [], 'y': ['x'], 'z': ['x', 'y'], 'c': ['a', 'b']}}, {'loop_count': 2, 'variable_dependencies': {}}, {'loop_count': 1, 'variable_dependencies': {'x': ['input']}}]


In [ ]:
import ast

def get_loop_count(tree):
    loop_count = 0
    for node in ast.walk(tree):
        if isinstance(node, (ast.For, ast.While)):
            loop_count += 1
    return loop_count

def get_variable_dependencies(tree):
    dependencies = {}
    for node in ast.walk(tree):
        if isinstance(node, ast.Assign):
            for target in node.targets:
                if isinstance(target, ast.Name):
                    dep = [n.id for n in ast.walk(node) if isinstance(n, ast.Name) and n.id != target.id]
                    dependencies[target.id] = dep
    return dependencies

solutions = [
    '''
x = 2
y = x + 1

for i in range(10):
    x= x+y
    print(x)
''',
    '''
y = 1
x = 0
while x<=10:
    x = x+y
    print(x)
''',
    '''
x,y = 0,1
for i in range(10):
    x,y = y,x+y
    print(x)
'''
]

dataset = []
for solution in solutions:
    tree = ast.parse(solution)
    loop_count = get_loop_count(tree)
    variable_dependencies = get_variable_dependencies(tree)
    dataset.append({'loop_count': loop_count, 'variable_dependencies': variable_dependencies})

print(dataset)


[{'loop_count': 1, 'variable_dependencies': {'x': ['y'], 'y': ['x']}}, {'loop_count': 1, 'variable_dependencies': {'y': [], 'x': ['y']}}, {'loop_count': 1, 'variable_dependencies': {}}]


In [ ]:
import ast

def get_loop_count(tree):
    count = 0
    for node in ast.walk(tree):
        if isinstance(node, (ast.For, ast.While)):
            count += 1
    return count

def get_variable_dependencies(tree):
    dependencies = {}
    for node in ast.walk(tree):
        if isinstance(node, ast.Assign):
            for target in node.targets:
                if isinstance(target, ast.Name):
                    dep = [n.id for n in ast.walk(node) if isinstance(n, ast.Name) and n.id != target.id]
                    dependencies[target.id] = dep
    return dependencies

def check_logic(tree):
    for node in ast.walk(tree):
        if isinstance(node, ast.For):
            if isinstance(node.iter, ast.Call) and node.iter.func.id == 'range':
                if isinstance(node.iter.args[0], ast.Num) and node.iter.args[0].n == 10:
                    return True
        if isinstance(node, ast.While):
            if isinstance(node.test, ast.Compare) and len(node.test.ops) == 1 and isinstance(node.test.ops[0], ast.LtE):
                if isinstance(node.test.left, ast.Name) and node.test.left.id == 'x' and isinstance(node.test.comparators[0], ast.Num) and node.test.comparators[0].n == 10:
                    return True
        if isinstance(node, ast.If):
            if isinstance(node.test, ast.Compare) and isinstance(node.body[0], ast.Break):
                return True
    return False

solutions = [
    '''
x = 2
y = x + 1

for i in range(10):
    x= x+y
    print(x)
''',
    '''
y = 1
x = 0
while x<=10:
    x = x+y
    print(x)
    if x>10:
        break
    for i in range(2):
        x=x+1
''',
    '''
x,y = 0,1
for i in range(10):
    x,y = y,x+y
    print(x)
    if x>10:
        break
''',
    '''
x = [1,2,3]
y = [4,5,6]
z = [7,8,9]

for i in range(len(x)):
    for j in range(len(y)):
        for k in range(len(z)):
            if x[i]+y[j]==z[k]:
                print(x[i], y[j], z[k])
'''
]


dataset = []
for solution in solutions:
    tree = ast.parse(solution)
    loop_count = get_loop_count(tree)
    variable_dependencies = get_variable_dependencies(tree)
    correct_logic = check_logic(tree)
    dataset.append({'loop_count': loop_count, 'variable_dependencies': variable_dependencies, 'correct_logic': correct_logic})

print(dataset)


[{'loop_count': 1, 'variable_dependencies': {'x': ['y'], 'y': ['x']}, 'correct_logic': True}, {'loop_count': 2, 'variable_dependencies': {'y': [], 'x': []}, 'correct_logic': True}, {'loop_count': 1, 'variable_dependencies': {}, 'correct_logic': True}, {'loop_count': 3, 'variable_dependencies': {'x': [], 'y': [], 'z': []}, 'correct_logic': False}]


In [ ]:
import ast

# def check_logic(tree):
#     for node in ast.walk(tree):
#         #Checking for a specific if condition and a specific function call
#         if isinstance(node, ast.If) and isinstance(node.test, ast.Compare) and isinstance(node.body[0], ast.Expr) and isinstance(node.body[0].value, ast.Call) :
#             #Checking for the specific function name and specific arguments passed to the function
#             if node.body[0].value.func.id == 'print' and isinstance(node.body[0].value.args[0], ast.BinOp) and isinstance(node.body[0].value.args[0].op, ast.Add):
#                 return True
#     return False

def check_logic(tree):
    for node in ast.walk(tree):
        if isinstance(node, ast.For):
            if isinstance(node.iter, ast.Call) and node.iter.func.id == 'range':
                if isinstance(node.iter.args[0], ast.Num) and node.iter.args[0].n == 10:
                    return True
        if isinstance(node, ast.While):
            if isinstance(node.test, ast.Compare) and len(node.test.ops) == 1 and isinstance(node.test.ops[0], ast.LtE):
                if isinstance(node.test.comparators[0], ast.Num) and node.test.comparators[0].n == 10:
                    return True
        if isinstance(node, ast.If):
            if isinstance(node.test, ast.Compare) and isinstance(node.body[0], ast.Break):
                return True
    return False


solutions1 = [
    '''
x = 2
y = x + 1
if x < y:
    print(x + y)
    x += 1
''',
    '''
x = 2
y = x + 1
if x > y:
    print(x - y)
    x += 1
''',
    '''
x = 2
y = x + 1

if x < y:
    print(x * y)
    x += 1
'''
]

solutions = [
    '''
x = 2
y = x + 1

for i in range(10):
    x= x+y
    print(x)
''',
    '''
y = 1
x = 0
while x<=10:
    x = x+y
    print(x)
    if x>10:
        break
    for i in range(2):
        x=x+1
''',
    '''
x,y = 0,1
for i in range(10):
    x,y = y,x+y
    print(x)
    if x>10:
        break
''',
    '''
x = [1,2,3]
y = [4,5,6]
z = [7,8,9]

for i in range(len(x)):
    for j in range(len(y)):
        for k in range(len(z)):
            if x[i]+y[j]==z[k]:
                print(x[i], y[j], z[k])
'''
]


dataset = []
for solution in solutions:
    tree = ast.parse(solution)
    loop_count = get_loop_count(tree)
    variable_dependencies = get_variable_dependencies(tree)
    correct_logic = check_logic(tree)
    dataset.append({'loop_count': loop_count, 'variable_dependencies': variable_dependencies, 'correct_logic': correct_logic})

print(dataset)


[{'loop_count': 1, 'variable_dependencies': {'x': ['y'], 'y': ['x']}, 'correct_logic': True}, {'loop_count': 2, 'variable_dependencies': {'y': [], 'x': []}, 'correct_logic': True}, {'loop_count': 1, 'variable_dependencies': {}, 'correct_logic': True}, {'loop_count': 3, 'variable_dependencies': {'x': [], 'y': [], 'z': []}, 'correct_logic': False}]


In [ ]:
import ast
from pydoc import locate

def get_loop_count(tree):
    loop_count = 0
    for node in ast.walk(tree):
        if isinstance(node, (ast.For, ast.While)):
            loop_count += 1
    return loop_count

def get_variable_dependencies(tree):
    dependencies = {}
    for node in ast.walk(tree):
        if isinstance(node, ast.Assign):
            for target in node.targets:
                if isinstance(target, ast.Name):
                    dep = [n.id for n in ast.walk(node) if isinstance(n, ast.Name) and n.id != target.id]
                    dependencies[target.id] = dep
    return dependencies

def check_logic(tree,expected_output):
    function_name = None
    function_args = None
    function_body = None
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            function_args = node.args
            function_body = node.body
    if function_name and function_args and function_body:
        func =  ast.fix_missing_locations(ast.Interactive(ast.FunctionDef(name=function_name,args=function_args,body=function_body,decorator_list=[],returns=None)))
        exec(compile(func,'','single'))
        func = locate(function_name)
        output = func(*function_args.args)
        if output == expected_output:
            return "Correct"
        else:
            return "Incorrect"
    else:
        return "Incomplete"

solutions = [
    ('x = 2\ny = x + 3\nz = x * y\n\ndef my_func(a, b):\n    c = a + b\n    return c',None),
    ('for i in range(10):\n    for j in range(5):\n        print(i * j)',None),
    ('while True:\n    x = input("Enter a number: ")\n    if x == "q":\n        break',None)
]
dataset = []
for solution,expected_output in solutions:
    tree = ast.parse(solution)
    loop_count = get_loop_count(tree)
    variable_dependencies = get_variable_dependencies(tree)
    logic = check_logic(tree,expected_output)
    dataset.append({'loop_count': loop_count, 'variable_dependencies': variable_dependencies, 'logic': logic})

print(dataset)


TypeError: ignored